In [1]:
from qiskit import *
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import minimize
df = pd.read_csv (r'Data.csv')
shots = 100

In [2]:
Y = list(df['Y'])
Theta = list(df['Theta'])
State = list(df['State'])

In [3]:
Y_Train_0 = Y[:500]
T_Train_0 = Theta[:500]
S_Train_0 = State[:500]

Y_Test_0 = Y[1000:]
T_Test_0 = Theta[1000:]
S_Test_0 = State[1000:]


for i in range(len(S_Train_0)):
    if S_Train_0[i] !=0:
       S_Train_0[i] =1
for i in range(len(S_Test_0)):
    if S_Test_0[i] !=0:
       S_Test_0[i] =1


In [38]:
def Circuit_0(Params,V):
    qc = QuantumCircuit(3,2)
    qc.h(range(2))
    qc.ry(V[0],0)
    qc.ry(V[1],1)
    qc.ccx(0,1,2)
    qc.ry((2.0*(np.pi - V[0])*(np.pi - V[1])),2)
    qc.ccx(0,1,2)
    for i in range(int(len(Params)/6)):
        qc.ry(Params[0+6*i],0)
        qc.ry(Params[1+6*i],1)
        qc.ry(Params[2+6*i],1)
        qc.cry(Params[3+6*i],0,1)
        qc.cry(Params[4+6*i],1,2)
        qc.cry(Params[5+6*i],2,0)
    qc.measure([0,1],[0,1])
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots)
    job_result = job.result()
    counts = job_result.get_counts(qc)

    A = list(job_result.get_counts(qc).keys())
    output = [0]*2
    for i in A:
        B = list(i)
        for j in range (2):
            if B[j] == '1':
                output[j]+= int(job_result.get_counts(qc).get(i)) 
    return (output)
    
        

In [36]:
Circuit_0([0]*6,[0,0])

[51, 49]

In [43]:
Params = np.random.uniform(low=-2.6, high=2.6, size=(60,))

In [44]:
def Cost_0(Params):
    cost = 0
    for i in range (len(Y_Train_0)):
        temp = [0]*2
        temp[S_Train_0[i]] = shots
        error = sum((np.array(Circuit_0(Params,[Y_Train_0[i],T_Train_0[i]]))-np.array(temp))**2)
        
        cost = cost+error
    return(cost)

def eff(Params):
    cost = 0
    for i in range (len(Y_Test_0)):
        output = Circuit_0(Params,[Y_Test_0[i],T_Test_0[i]])
        if output.index(max(output)) == S_Test_0[i]:
            cost = cost+1
    return(cost/len(Y_Test_0))

In [ ]:
minimum_0 = minimize(Cost_0, Params, method="COBYLA", tol=0.001)
minimum_0

In [ ]:
eff(minimum_0.x)